## Use \_\_getattr\_\_ for dynamic attribute resolution

In [1]:
class DictWrapper:

    def __init__(self, dict_contents: dict):
        self.dict_contents = dict_contents
        
    def __getattr__(self, attr):
        try:
            return self.dict_contents[attr]
        except KeyError:
            raise AttributeError

In [2]:
test_dict: dict = {
    'my_attribute': 'my_value'
}
dict_obj = DictWrapper(test_dict)

In [3]:
assert dict_obj.my_attribute == 'my_value'

In [4]:
try:
    dict_obj.not_an_attribute
except AttributeError:
    print('Received expected AttributeError!')
else:
    assert False, 'Did not receieve expected AttributeError!'

Received expected AttributeError!


## Object assignment references original object

In [5]:
class DummyClass: pass

In [6]:
dummy = DummyClass()
same_dummy = dummy
assert dummy is same_dummy

## Use copy to create a copy of an object instead of a reference

In [7]:
import copy

copy_dummy = copy.copy(dummy)
assert dummy is not copy_dummy

## User \_\_enter\_\_ and \_\_exit\_\_ to create a context manager

In [8]:
import datetime
import time


class ContextTime:
    
    def __init__(self):
        pass
    
    def __enter__(self):
        self.enter_time = datetime.datetime.now()
        str_enter_time = self.enter_time.strftime("%H:%M:%S")
        print(f"Entered context manager at {str_enter_time}")
        
    def __exit__(self, exception_type, exception_value, traceback):
        self.exit_time = datetime.datetime.now()
        str_exit_time = self.exit_time.strftime("%H:%M:%S")
        print(f"Exited context manager at {str_exit_time}")
        time_in_context_manager = self.exit_time - self.enter_time
        print(f"Spent {time_in_context_manager.seconds} second(s) in context manager")

In [9]:
with ContextTime():
    time.sleep(2)

Entered context manager at 22:37:22
Exited context manager at 22:37:24
Spent 2 second(s) in context manager


## Use decorators to resolve class attributes

Note: this is neat but may or may not always be a good idea. The idea is that a function is meant to be called separately from \_\_init\_\_ that sets some important class attribute. If another method goes to use that function and does not have the attribute set, it can attempt to set it itself.

In [10]:
import functools
import psutil

def resolve_python_pid(func):
    """Tries resolving python's process id if not already set as self._py_pid
    
    First checks if self._py_pid is None in case python was not open using
    open_py() method. If self._py_pid is not set, uses psutil process_iter to
    iterate over running processes and determine if there is single python
    instance running. If so, it uses the pid of that instance of python. If 
    there are 0 or multiple instances of python, it prints an error message.
    """

    @functools.wraps(func)
    def wrapper(self, *args, **kwargs):
    
        if self._py_pid is None:
            py_pids = [
                proc.pid for proc in psutil.process_iter() if self._py_proc_name in proc.name()
            ]
            assert len(py_pids) == 1, (
                f'Could not resolve process ID for python. This function was '
                f'called without first calling open_py() and attempted to resolve '
                f'process ID by searching through running processes but there are '
                f'{len(py_pids)} instances of python running.'
            )
            
            self._py_pid = py_pids[0]

        return func(self, *args, **kwargs)
    return wrapper

In [11]:
import subprocess
import sys

class PyLauncher:

    def __init__(self):
        self._py_path = sys.executable
        self._py_proc_name = 'python'
        self._py_pid = None

    def open_py(self):
        py_ = subprocess.Popen([self._py_path])
        self._py_pid = py_.pid

    @resolve_python_pid
    def print_py_pid(self):
        print(self._py_pid)

In [12]:
pylaunch = PyLauncher()
try:
    pylaunch.print_py_pid()
except AssertionError as e:
    print(e)

Could not resolve process ID for python. This function was called without first calling open_py() and attempted to resolve process ID by searching through running processes but there are 4 instances of python running.


In [13]:
pylaunch.open_py()
pylaunch.print_py_pid()

15956
